In [1]:
from keras.applications.vgg16 import VGG16
import os
from glob import glob
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model
from keras.applications.densenet import DenseNet201
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
import keras.backend as K
from keras.models import load_model
%matplotlib inline
import matplotlib.pyplot as plt
import gc

In [2]:
pre_trained_model= tf.keras.applications.InceptionV3(input_shape=(224,224,3), include_top=False, weights="imagenet")

87924736/87910968 [==============================] - 0s 0us/step


In [3]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [4]:
x = layers.GlobalMaxPooling2D()(pre_trained_model.output)
x = layers.Dense(512, activation='relu')(x)
# Add a dropout rate of 0.7
x = layers.Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(7, activation='softmax')(x)

In [5]:
model = Model(pre_trained_model.input, x)
optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [7]:
X = np.load("/content/drive/MyDrive/All datasets/skin disease/224_224_X_train.npy")
y=np.load("/content/drive/MyDrive/All datasets/skin disease/224_224_y_train.npy")

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33333, random_state=42)

In [9]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

In [10]:
train_datagen = ImageDataGenerator(rotation_range=60, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, fill_mode='nearest')

train_datagen.fit(X_train)

val_datagen = ImageDataGenerator()
val_datagen.fit(X_val)

In [11]:
batch_size = 32
epochs = 3
history = model.fit(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size), 
                              validation_steps=(X_val.shape[0] // batch_size))

Epoch 1/3
156/156 [==============================] - 77s 385ms/step - loss: 1.5168 - accuracy: 0.6060 - val_loss: 0.9396 - val_accuracy: 0.6939
Epoch 2/3
156/156 [==============================] - 58s 371ms/step - loss: 1.0391 - accuracy: 0.6663 - val_loss: 0.8930 - val_accuracy: 0.6995
Epoch 3/3
156/156 [==============================] - 57s 365ms/step - loss: 0.9694 - accuracy: 0.6764 - val_loss: 0.8716 - val_accuracy: 0.6911


In [12]:
for layer in model.layers:
    layer.trainable = True

In [13]:
optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])

In [14]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=2)

In [15]:
batch_size = 32
epochs = 40
history = model.fit(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size),
                              callbacks=[learning_rate_reduction])

Epoch 1/40
156/156 [==============================] - 71s 390ms/step - loss: 0.9335 - acc: 0.6806 - val_loss: 1.2275 - val_acc: 0.6911 - lr: 1.0000e-04
Epoch 2/40
156/156 [==============================] - 58s 370ms/step - loss: 0.7635 - acc: 0.7282 - val_loss: 0.7111 - val_acc: 0.7552 - lr: 1.0000e-04
Epoch 3/40
156/156 [==============================] - 57s 366ms/step - loss: 0.6530 - acc: 0.7660 - val_loss: 0.5700 - val_acc: 0.7873 - lr: 1.0000e-04
Epoch 4/40
156/156 [==============================] - 57s 365ms/step - loss: 0.5615 - acc: 0.7984 - val_loss: 0.5827 - val_acc: 0.8121 - lr: 1.0000e-04
Epoch 5/40
156/156 [==============================] - 57s 364ms/step - loss: 0.5075 - acc: 0.8149 - val_loss: 0.5574 - val_acc: 0.7857 - lr: 1.0000e-04
Epoch 6/40
156/156 [==============================] - 57s 366ms/step - loss: 0.4624 - acc: 0.8310 - val_loss: 0.4781 - val_acc: 0.8337 - lr: 1.0000e-04
Epoch 7/40
156/156 [==============================] - 57s 365ms/step - loss: 0.4276 - ac

In [16]:
loss_val, acc_val = model.evaluate(X_val, y_val, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))

79/79 [==============================] - 8s 87ms/step - loss: 0.6209 - acc: 0.8762
Validation: accuracy = 0.876198  ;  loss_v = 0.620934


In [17]:
X_test=np.load("/content/drive/MyDrive/All datasets/skin disease/X_test.npy")
y_test=np.load("/content/drive/MyDrive/All datasets/skin disease/y_test.npy")

In [18]:
y_test = to_categorical(y_test)

In [19]:
loss_test, acc_test = model.evaluate(X_test, y_test, verbose=1)
print("Test: accuracy = %f  ;  loss = %f" % (acc_test, loss_test))

79/79 [==============================] - 6s 76ms/step - loss: 0.7494 - acc: 0.8554
Test: accuracy = 0.855431  ;  loss = 0.749447


In [20]:
model.save("/content/drive/MyDrive/All datasets/skin disease/skin disease with inceptionv3.h5")